**Imports**


In [132]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [133]:
from fastai import *
from fastai.text import *

In [134]:
import sklearn.feature_extraction.text as sklearn_text

For help it is possible to use ??


In [135]:
?? URLs

In [136]:

path = untar_data(URLs.IMDB_SAMPLE)
path

WindowsPath('C:/Users/Arles/.fastai/data/imdb_sample')

Load to ger a sense of how the data looks like:

In [137]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


TextList of fast.ai

In [138]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols = 'text').split_from_df(col=2).label_from_df(cols=0))

**Exploring what our data looks like**

Tokenization and numericalization



    UNK (xxunk) is for an unknown word (one that isn't present in the current vocabulary)
    PAD (xxpad) is the token used for padding, if we need to regroup several texts of different lengths in a batch
    BOS (xxbos) represents the beginning of a text in your dataset
    FLD (xxfld) is used if you set mark_fields=True in your TokenizeProcessor to separate the different fields of texts (if your texts are loaded from several columns in a dataframe)
    TK_MAJ (xxmaj) is used to indicate the next word begins with a capital in the original text
    TK_UP (xxup) is used to indicate the next word is written in all caps in the original text
    TK_REP (xxrep) is used to indicate the next character is repeated n times in the original text (usage xxrep n {char})
    TK_WREP(xxwrep) is used to indicate the next word is repeated n times in the original text (usage xxwrep n {word})


In [139]:
movie_reviews.valid.x[0], movie_reviews.valid.y[0]

(Text xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
  
   xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
  
   xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , xxmaj xxunk xxmaj xxunk ,

In [140]:
len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

In [141]:
len(movie_reviews.vocab.itos), len(movie_reviews.vocab.stoi)

(6008, 19161)

In [142]:
movie_reviews.vocab.itos[220:230]

['bit',
 'might',
 'things',
 'horror',
 'us',
 'almost',
 'may',
 'right',
 'must',
 'away']

In [143]:
movie_reviews.vocab.stoi['almost']

225

In [144]:
movie_reviews.vocab.itos[225]

'almost'

In [145]:
movie_reviews.vocab.stoi

defaultdict(int,
            {'xxunk': 0,
             'xxpad': 1,
             'xxbos': 2,
             'xxeos': 3,
             'xxfld': 4,
             'xxmaj': 5,
             'xxup': 6,
             'xxrep': 7,
             'xxwrep': 8,
             'the': 9,
             '.': 10,
             ',': 11,
             'and': 12,
             'a': 13,
             'of': 14,
             'to': 15,
             'is': 16,
             'it': 17,
             'in': 18,
             'i': 19,
             'that': 20,
             'this': 21,
             '"': 22,
             "'s": 23,
             '\n \n ': 24,
             '-': 25,
             'was': 26,
             'as': 27,
             'for': 28,
             'movie': 29,
             'with': 30,
             'but': 31,
             'film': 32,
             'you': 33,
             ')': 34,
             'on': 35,
             '(': 36,
             "n't": 37,
             'are': 38,
             'he': 39,
             'his': 40,
       

In [146]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi['arles']]

'xxunk'

In [147]:
movie_reviews.vocab.stoi['arles']


0

**Create our term-documment matrix**


Counter: data structure is a dictionary of frequencies


In [148]:
c = Counter([4,2,8,8,4,8])

In [149]:
c

Counter({4: 2, 2: 1, 8: 3})

In [150]:
c.keys()

dict_keys([4, 2, 8])

In [151]:
c.values()

dict_values([2, 1, 3])

Counters are from collections module

**Sparse Matrices**

-> coordinate-wise storage

In [152]:
(movie_reviews.valid.x)[0].data

array([ 2,  5, 21, 71, ..., 15,  5,  0, 52], dtype=int64)

In [153]:
(movie_reviews.valid.x)[0]

Text xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
 
  xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
 
  xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , xxmaj xxunk xxmaj xxunk , xxma

In [154]:
movie_reviews.vocab.itos[189]

'funny'

In [155]:
feature_counter = Counter((movie_reviews.valid.x)[0].data)
feature_counter

Counter({2: 1,
         5: 32,
         21: 3,
         71: 1,
         189: 1,
         748: 1,
         288: 1,
         285: 1,
         63: 2,
         221: 1,
         666: 2,
         59: 1,
         13: 4,
         2705: 1,
         14: 6,
         2875: 1,
         11: 10,
         18: 2,
         358: 1,
         0: 32,
         77: 1,
         15: 6,
         478: 1,
         1833: 1,
         50: 3,
         9: 10,
         319: 1,
         6: 1,
         2743: 1,
         12: 1,
         115: 1,
         4126: 1,
         197: 2,
         1331: 1,
         25: 2,
         324: 1,
         10: 7,
         3963: 1,
         16: 4,
         74: 1,
         24: 3,
         2817: 1,
         5821: 1,
         2595: 1,
         710: 1,
         3429: 1,
         84: 1,
         149: 1,
         20: 1,
         26: 1,
         605: 1,
         378: 1,
         1057: 1,
         251: 1,
         258: 1,
         1346: 1,
         194: 1,
         239: 1,
         49: 1,
         27

In [156]:

def get_term_doc_matrix(label_list, vocab_len):
    j_indices = []
    indptr = []
    values = []
    indptr.append(0)

    for i, doc in enumerate(label_list):
        feature_counter = Counter(doc.data)
        j_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        indptr.append(len(j_indices))
        
#     return (values, j_indices, indptr)

    return scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, vocab_len),
                                   dtype=int)



In [157]:
%%time
val_term_doc = get_term_doc_matrix(movie_reviews.valid.x, len(movie_reviews.vocab.itos))

Wall time: 34.9 ms


In [158]:
%%time
trn_term_doc = get_term_doc_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))

Wall time: 126 ms


In [159]:
trn_term_doc.shape

(800, 6008)

In [160]:
val_term_doc.shape

(200, 6008)

In [161]:
trn_term_doc[:10, :10]

<10x10 sparse matrix of type '<class 'numpy.int32'>'
	with 45 stored elements in Compressed Sparse Row format>

**More data exploration**



Obtain a dense version of sparse matrix.

In [162]:
val_term_doc.todense()[:10, :10]

matrix([[32,  0,  1,  0, ...,  1,  0,  0, 10],
        [ 9,  0,  1,  0, ...,  1,  0,  0,  7],
        [ 6,  0,  1,  0, ...,  0,  0,  0, 12],
        [78,  0,  1,  0, ...,  0,  0,  0, 44],
        ...,
        [ 8,  0,  1,  0, ...,  0,  0,  0,  8],
        [43,  0,  1,  0, ...,  8,  1,  0, 25],
        [ 7,  0,  1,  0, ...,  1,  0,  0,  9],
        [19,  0,  1,  0, ...,  2,  0,  0,  5]])

In [163]:
movie_reviews.vocab.itos[-1:]

['sollett']

In [164]:
movie_reviews.vocab.itos[0]

'xxunk'

In [165]:
movie_reviews.vocab.stoi['sollett']

6007

In [166]:
review = movie_reviews.valid.x[1]
review

Text xxbos i saw this movie once as a kid on the late - late show and fell in love with it . 
 
  xxmaj it took 30 + years , but i recently did find it on xxup dvd - it was n't cheap , either - in a xxunk that xxunk in war movies . xxmaj we watched it last night for the first time . xxmaj the audio was good , however it was grainy and had the trailers between xxunk . xxmaj even so , it was better than i remembered it . i was also impressed at how true it was to the play . 
 
  xxmaj the xxunk is around here xxunk . xxmaj if you 're xxunk in finding it , fire me a xxunk and i 'll see if i can get you the xxunk . xxunk

*Exercise:* confirm a value of 2 is stored in term document matrix for the row corresponding to this review and the column corresponding to the word late.


In [167]:
movie_reviews.vocab.stoi['late']

451

In [168]:
val_term_doc[1, 451]

2

*Excercise:* How could you convert review data back to text without using review.text

In [259]:
[movie_reviews.vocab.itos[a] for a in review.data]

['xxbos',
 'xxunk',
 'is',
 'normally',
 'xxunk',
 ',',
 'and',
 'it',
 "'s",
 'never',
 'really',
 'bothered',
 'me',
 'before',
 '--',
 'i',
 'loved',
 '"',
 'xxunk',
 '2',
 '"',
 ',',
 'but',
 'i',
 'really',
 'do',
 "n't",
 'like',
 'pseudo',
 'xxunk',
 'or',
 'playing',
 'on',
 'xxunk',
 '.',
 'the',
 'whole',
 'movie',
 'is',
 'based',
 'on',
 'a',
 'man',
 'with',
 'esp',
 'caused',
 'by',
 'an',
 'xxunk',
 'accident',
 'and',
 'a',
 'witch',
 '.',
 'i',
 "'m",
 'not',
 'opposed',
 'to',
 'xxunk',
 ',',
 'and',
 'i',
 'liked',
 '"',
 'carrie',
 '"',
 '(',
 'the',
 'novel',
 'and',
 'the',
 'movie',
 ')',
 'but',
 'this',
 'one',
 'bothered',
 'me',
 '.',
 'i',
 'think',
 'it',
 "'s",
 'because',
 'of',
 'the',
 'main',
 'character',
 'developing',
 'esp',
 'from',
 'an',
 'electric',
 'line',
 '.',
 'also',
 'the',
 'university',
 'professor',
 'wasting',
 'his',
 'time',
 'studying',
 'esp',
 'cases',
 '.',
 'i',
 'was',
 "n't",
 'alive',
 'in',
 'the',
 '60s',
 'to',
 'know',




Exercise: Confirm that review has 81 distinct tokens


In [174]:
len(set(review.data))


81

In [169]:
review.data[:20]

array([   2,   19,  248,   21,   29,  327,   27,   13,  618,   35,    9,  451,   25,  451,  149,   12, 1065,   18,
        142,   30], dtype=int64)

In [170]:
val_term_doc[1,451]


2

In [171]:
val_term_doc[1].sum()

144

In [172]:
val_term_doc.todense()[1,451]

2

***Naive Bayes***


We define the logcount ratio r of each word f

In [260]:
movie_reviews.y.classes

['negative', 'positive']

In [261]:
x = trn_term_doc
y = movie_reviews.train.y
val_y = movie_reviews.valid.y

In [262]:
positive = y.c2i['positive']
negative = y.c2i['negative']

In [263]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0))) # 
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [264]:
len(p0)

38456

In [265]:
len(p1)

38456

In [266]:
np.squeeze(np.asarray(x[y.items==negative].sum(0)))

array([882,   0, 417,   0, ...,   0,   0,   0,   0], dtype=int32)

In [269]:
type(x[y.items==negative])

scipy.sparse.csr.csr_matrix

In [268]:
np.asarray(x[y.items==negative].sum(0))

array([[882,   0, 417,   0, ...,   0,   0,   0,   0]], dtype=int32)

In [267]:
np.squeeze(np.asarray(x[y.items==positive].sum(0)))

array([843,   0, 383,   0, ...,   0,   0,   0,   0], dtype=int32)

In [183]:
v = movie_reviews.vocab

In [184]:
v.itos[0]

'xxunk'

Word loved in positive reviews

In [271]:
p0[v.stoi['loved']], p1[v.stoi['loved']]

(8, 16)

word hated in reviews

In [272]:
p0[v.stoi['hated']], p1[v.stoi['hated']]

(2, 7)

In [188]:
v.stoi['hated']

1977

In [189]:
a = np.argwhere((x[:,1977]) > 0)[:,0]; a

array([ 15,  49, 304, 351, 393, 612, 695, 773])

In [190]:
b = np.argwhere(y.items==positive)[:,0]; b

array([  1,   3,  10,  11, ..., 787, 789, 790, 797], dtype=int64)

In [191]:
set(a).intersection(set(b))

{393, 612, 695}

In [192]:

review = movie_reviews.train.x[695]
review.text

"xxbos xxmaj xxunk , yeah this episode is extremely underrated . \n \n  xxmaj even though there is a xxup lot of bad writing and acting at parts . i think the good over wins the bad . \n \n  i love the xxunk parts and the big ' twist ' at the end . i absolutely love that scene when xxmaj michelle xxunk xxmaj tony . xxmaj it 's actually one of my favorite scenes of xxmaj season 1 . \n \n  xxmaj for some reason , people have always hated the xxmaj xxunk episodes , yet i have always liked them . xxmaj they 're not the best , in terms of writing . but the theme really does interest me , \n \n  i 'm gon na give it a xxup three star , but if the writing were a little more consistent i 'd give it xxup four ."

Negative reviews of the word loved

In [193]:
v.stoi['loved']

535

In [194]:
a = np.argwhere((x[:,535]) > 0)[:,0]; a

array([  1,  15,  29,  69,  75,  79, 174, 185, 200, 205, 262, 296, 303, 333, 350, 351, 398, 407, 440, 489, 496, 528,
       538, 600, 602, 605, 627, 642, 657, 660, 700, 712, 729, 735, 755, 767, 785])

In [195]:
b = np.argwhere(y.items==negative)[:,0]; b

array([  0,   2,   4,   5, ..., 795, 796, 798, 799], dtype=int64)

In [196]:
set(a).intersection(set(b))

{15, 200, 205, 303, 351, 398, 600, 605, 642, 700, 729, 767}

In [197]:
review = movie_reviews.train.x[205]
review.text

'xxbos xxunk is normally xxunk , and it \'s never really bothered me before -- i loved " xxunk 2 " , but i really do n\'t like pseudo xxunk or playing on xxunk . the whole movie is based on a man with esp caused by an xxunk accident and a witch . i \'m not opposed to xxunk , and i liked " carrie " ( the novel and the movie ) but this one bothered me . i think it \'s because of the main character developing esp from an electric line . also the university professor wasting his time studying esp cases . i was n\'t alive in the 60s to know first hand whether or not esp was a common xxunk then , but i assume that any theory of such nature would simply be xxunk . what really bothered me was the way the police were xxunk xxunk as xxunk , xxunk villains who can seem like normal people 99 % of the time and then just snap . nothing could be further from the truth . i xxunk such xxunk because they add to public xxunk . many people still think that xxunk is xxunk identity xxunk . whoever wrote thi

*Applying Naive Bayes*

In [198]:
pr1 = (p1+1) / ((y.items == positive).sum() +1)

In [199]:
pr0 = (p0+1) / ((y.items == negative).sum() +1)


In [200]:
r = np.log(pr1/pr0); r

array([-0.015487,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

**Vocabulary most likely associated with positive or negative reviews**

In [201]:
biggest = np.argpartition(r, -10)[-10:]

In [202]:
smallest = np.argpartition(r, 10)[:10]

most positive words:

In [203]:
[v.itos[k] for k in biggest]

['sport',
 'davies',
 'gilliam',
 'fanfan',
 'biko',
 'felix',
 'noir',
 'jabba',
 'astaire',
 'jimmy']

In [204]:
np.argmax(trn_term_doc[:,v.stoi['biko']])

515

In [205]:
movie_reviews.train.x[515]

Text xxbos " xxmaj the xxmaj true xxmaj story xxmaj of xxmaj the xxmaj friendship xxmaj that xxmaj shook xxmaj south xxmaj africa xxmaj and xxmaj xxunk xxmaj the xxmaj world . " 
 
  xxmaj richard xxmaj attenborough , who directed " a xxmaj bridge xxmaj too xxmaj far " and " xxmaj gandhi " , wanted to bring the story of xxmaj steve xxmaj biko to life , and the journey and trouble that xxunk xxmaj donald xxmaj woods went through in order to get his story told . xxmaj the films uses xxmaj wood 's two books for it 's information and basis - " xxmaj biko " and " xxmaj asking for xxmaj trouble " . 
 
  xxmaj the film takes place in the late 1970 's , in xxmaj south xxmaj africa . xxmaj south xxmaj africa is in the grip of the terrible apartheid , which keeps the blacks separated from the whites and xxunk the whites as the superior race . xxmaj the blacks are forced to live in xxunk on the xxunk of the cities and xxunk , and they come under frequent xxunk by the police and the army . xxmaj w

In [206]:
[v.itos[k] for k in smallest]

['worst',
 'crap',
 'crater',
 'porn',
 'disappointment',
 'dog',
 'vargas',
 'naschy',
 'fuqua',
 'soderbergh']

In [207]:
np.argmax(trn_term_doc[:,v.stoi['soderbergh']])

434

In [208]:
movie_reviews.train.x[434]

Text xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of " xxmaj at xxmaj the xxmaj movies " in taking xxmaj steven xxmaj soderbergh to task . 
 
  xxmaj it 's usually satisfying to watch a film director change his style / subject , but xxmaj soderbergh 's most recent stinker , xxmaj the xxmaj girlfriend xxmaj xxunk ) , was also missing a story , so narrative ( and editing ? ) seem to suddenly be xxmaj soderbergh 's main challenge . xxmaj strange , after xxunk years in the business . xxmaj he was probably never much good at narrative , just xxunk it well inside " edgy " projects . 
 
  xxmaj none of this excuses him this present , almost diabolical failure . xxmaj as xxmaj david xxmaj xxunk xxunk , " two parts of xxmaj che do n't ( even ) make a whole " . 
 
  xxmaj epic xxunk in name only , xxmaj che(2008 ) barely qualifies as a feature film 

Continuing to Naive Bayes

In [209]:
(y.items == positive).mean(), (y.items == negative).mean(), 

(0.47875, 0.52125)

- Greater than 0 positive
- less than 0 negative


In [210]:
b = np.log((y.items==positive).mean()/(y.items==negative).mean())
b

-0.08505123261815539

In [211]:
preds = (val_term_doc@r + b) > 0
preds

array([ True, False, False, False, ...,  True, False, False,  True])

In [212]:
(preds == val_y.items).mean()

0.645

**Processing Full Data Set**

Download:

In [213]:
path = untar_data(URLs.IMDB)
path.ls()

[WindowsPath('C:/Users/Arles/.fastai/data/imdb/imdb.vocab'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/imdb_textlist_class'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/lm_databunch'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/models'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/README'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/test'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/tmp_clas'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/tmp_lm'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/train'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/unsup')]

In [214]:
(path/'train').ls()

[WindowsPath('C:/Users/Arles/.fastai/data/imdb/train/labeledBow.feat'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/train/neg'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/train/pos'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/train/unsupBow.feat')]

In [215]:
reviews_full = (TextList.from_folder(path)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos']))
             #label them all with their folders

In [216]:
len(reviews_full.train), len(reviews_full.valid)

(25000, 25000)

In [217]:
v = reviews_full.vocab

In [218]:
v.itos[100:110]

['bad',
 'people',
 'will',
 'other',
 'also',
 'into',
 'first',
 'because',
 'great',
 'how']

In [219]:
%%time
val_term_doc = get_term_doc_matrix(reviews_full.valid.x, len(reviews_full.vocab.itos))

Wall time: 4.18 s


In [220]:
%%time
trn_term_doc = get_term_doc_matrix(reviews_full.train.x, len(reviews_full.vocab.itos))

Wall time: 4.82 s


**Save data**

Matrices can be saved

In [221]:
scipy.sparse.save_npz("trn_term_doc.npz", trn_term_doc)

In [222]:
scipy.sparse.save_npz("val_term_doc.npz", val_term_doc)

In [223]:
trn_term_doc = scipy.sparse.load_npz("trn_term_doc.npz")
val_term_doc = scipy.sparse.load_npz("val_term_doc.npz")

Naive Bayes on Full dataset

In [224]:
x=trn_term_doc
y=reviews_full.train.y

val_y = reviews_full.valid.y.items

In [225]:
x

<25000x38456 sparse matrix of type '<class 'numpy.int32'>'
	with 3716265 stored elements in Compressed Sparse Row format>

In [226]:
positive = y.c2i['pos']
negative = y.c2i['neg']

In [227]:
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))

In [228]:
p1[:20]

array([ 28450,      0,  12500,      0,      0, 342619,  20464,   1338,      7, 173122, 138000, 143763,  89570,  83404,
        76828,  66715,  58510,  47896,  50177,  40451], dtype=int32)

**Data exploration**

Bigger ratio mean the word is indicative of a negative review, smaller ratios (<1) mean a positive review.

In [229]:


def neg_pos_given_word(word):
    print(p0[v.stoi[word]]/p1[v.stoi[word]])



In [230]:
neg_pos_given_word('hated')

2.051546391752577


In [231]:
neg_pos_given_word('liked')

0.6424702058504875


In [232]:
neg_pos_given_word('loved')

0.3139963167587477


In [233]:
neg_pos_given_word('best')

0.48538961038961037


In [234]:
neg_pos_given_word('worst')

9.837301587301587


In [235]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [236]:
r = np.log(pr1/pr0)

In [237]:
r[v.stoi['hated']]

-0.7133498878774648

In [238]:
r[v.stoi['loved']]

1.1563661500586044

In [239]:
r[v.stoi['worst']]

-2.2826243504315076

In [240]:
r[v.stoi['best']]

0.7225576052173609

Back to naive bayes

In [241]:

negative = y.c2i['neg']
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [242]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [243]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean()); b

0.0

In [244]:
preds = (val_term_doc @ r + b) > 0

In [245]:
(preds == val_y).mean()

0.8084

**Binarized naive bayes**

Only matters whether a word is in the review or not (not the frequency of the word):

In [246]:
x=trn_term_doc.sign()
y=reviews_full.train.y

In [247]:
x.todense()[:10,:10]

matrix([[1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [0, 0, 1, 0, ..., 0, 1, 0, 1],
        ...,
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [0, 0, 1, 0, ..., 1, 0, 0, 1],
        [0, 0, 1, 0, ..., 0, 0, 0, 1],
        [0, 0, 1, 0, ..., 1, 0, 0, 1]])

In [248]:
negative = y.c2i['neg']
positive = y.c2i['pos']

In [249]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [250]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [251]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

preds = (val_term_doc.sign() @ r + b) > 0

In [252]:
(preds==val_y).mean()

0.82924

**Logistic Regression**

unigrams (one word)

In [253]:
from sklearn.linear_model import LogisticRegression

In [254]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y.items.astype(int))
preds = m.predict(val_term_doc)
(preds == val_y).mean()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7704

In [255]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y.items.astype(int))
preds = m.predict(val_term_doc.sign())
(preds == val_y).mean()

0.88536

**Trigram with NB features**



Our next model is a version of logistic regression with Naive Bayes features described here. For every document we compute binarized features as described above, but this time we use bigrams and trigrams too. Each feature is a log-count ratio. A logistic regression model is then trained to predict sentiment.


**ngrams**

An n-gram is a contiguous sequence of n items (where the items can be characters, syllables, or words). A 1-gram is a unigram, a 2-gram is a bigram, and a 3-gram is a trigram.

Here, we are referring to sequences of words. So examples of bigrams include "the dog", "said that", and "can't you".


In [256]:
path = untar_data(URLs.IMDB_SAMPLE)

In [257]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0))

In [258]:
x=train_ngram_doc_matrix.sign()
val_x=valid_ngram_doc_matrix.sign()
y=movie_reviews.train.y

NameError: name 'train_ngram_doc_matrix' is not defined

In [ ]:
v = movie_reviews.vocab.itos

In [ ]:
vocab_len = len(v)

**Our data**

*Create train matrix*

In [ ]:
min_n=1
max_n=3

j_indices = []
indptr = []
values = []
indptr.append(0)
num_tokens = vocab_len

itongram = dict()
ngramtoi = dict()

Following creates n-grams

In [ ]:
for i, doc in enumerate(movie_reviews.train.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) not in ngramtoi:
                if len(ngram)==1:
                    num = ngram[0]
                    ngramtoi[str(ngram)] = num
                    itongram[num] = ngram
                else:
                    ngramtoi[str(ngram)] = num_tokens
                    itongram[num_tokens] = ngram
                    num_tokens += 1
            this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

In [ ]:
train_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [ ]:
train_ngram_doc_matrix

**Looking at our data**



In [ ]:
len(ngramtoi), len(itongram)

In [ ]:
itongram[20005]

In [ ]:
ngramtoi[str(np.array([15, 9, 710]))]

In [ ]:
itongram[100000]

In [ ]:
v[1140], v[33]

In [ ]:
itongram[200000]


In [ ]:
itongram[6116]

In [ ]:
v[85], v[191], v[64]

In [ ]:
itongram[80000]

In [ ]:
v[2594], v[14], v[2618]

*Create valid matrix*

In [ ]:
j_indices = []
indptr = []
values = []
indptr.append(0)

for i, doc in enumerate(movie_reviews.valid.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) in ngramtoi:
                this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

In [ ]:
valid_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [ ]:
valid_ngram_doc_matrix

In [ ]:
train_ngram_doc_matrix

*Save data*

In [ ]:
scipy.sparse.save_npz("train_ngram_matrix.npz", train_ngram_doc_matrix)

In [ ]:
scipy.sparse.save_npz("valid_ngram_matrix.npz", valid_ngram_doc_matrix)

In [ ]:
with open('itongram.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ngramtoi.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)


Load Data

In [ ]:
train_ngram_doc_matrix = scipy.sparse.load_npz("train_ngram_matrix.npz")
valid_ngram_doc_matrix = scipy.sparse.load_npz("valid_ngram_matrix.npz")

In [ ]:
with open('itongram.pickle', 'rb') as handle:
    b = pickle.load(handle)
    
with open('ngramtoi.pickle', 'rb') as handle:
    b = pickle.load(handle)

Naive Bayes

In [ ]:
x=train_ngram_doc_matrix
y=movie_reviews.train.y

In [ ]:
positive = y.c2i['positive']
negative = y.c2i['negative']

In [ ]:
x

In [ ]:
k=260374

In [ ]:
pos = (y.items == positive)[:k]
neg = (y.items == negative)[:k]

In [ ]:
xx = x[:k]

In [ ]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [ ]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [ ]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [ ]:
r = np.log(pr1/pr0)


In [ ]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())
b

In [ ]:
(y.items==positive).mean(), (y.items==negative).mean()

In [ ]:
pre_preds = valid_ngram_doc_matrix @ r.T + b

pre_preds

In [ ]:
preds = pre_preds.T>0

In [ ]:
preds[:10]

In [ ]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [ ]:
(preds == valid_labels).mean()

**Binarized Naive Bayes**

In [ ]:
trn_x_ngram_sgn = train_ngram_doc_matrix.sign()
val_x_ngram_sgn = valid_ngram_doc_matrix.sign()

In [ ]:
xx = trn_x_ngram_sgn[:k]

In [ ]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [ ]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [ ]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

pre_preds = val_x_ngram_sgn @ r.T + b
preds = pre_preds.T>0

In [ ]:
(preds==valid_labels).mean()

*Logistic Regresion*

In [ ]:
from sklearn.linear_model import LogisticRegression

use CountVectorizer to compare

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
veczr = CountVectorizer(ngram_range=(1,3), preprocessor=noop, tokenizer=noop, max_features=800000)

In [ ]:
docs = movie_reviews.train.x

In [ ]:
train_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.train.x]

In [ ]:
valid_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.valid.x]

In [ ]:
%%time
train_ngram_doc = veczr.fit_transform(train_words)

In [ ]:
train_ngram_doc

In [ ]:
veczr.vocabulary_

In [ ]:
val_ngram_doc = veczr.transform(valid_words)

In [ ]:
val_ngram_doc

In [ ]:
vocab = veczr.get_feature_names()

In [ ]:
vocab[200000:200005]

Binarized Naive Bayes, using ngrams from CountVectorizer

In [ ]:
y=movie_reviews.train.y

C is the inverse of regularization strength; smaller values specify stronger regularization. Regularized:

In [ ]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(train_ngram_doc.sign(), y.items);

preds = m.predict(val_ngram_doc.sign())
(preds.T==valid_labels).mean()

Not binarized

In [ ]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(train_ngram_doc, y.items);

preds = m.predict(val_ngram_doc)
(preds.T==valid_labels).mean()

In [ ]:
preds[:10]

With custom ngrams

In [ ]:
m2 = LogisticRegression(C=0.1, dual=True)
m2.fit(trn_x_ngram_sgn, y.items)

In [ ]:
preds = m2.predict(val_x_ngram_sgn)
(preds.T==valid_labels).mean()

Worse performance when not binarized. I manually tried several different C values, and this was the best I found:

Following process takes time:

In [ ]:
m2 = LogisticRegression(C=0.0001, dual=True, max_iter=50000)
m2.fit(train_ngram_doc_matrix, y.items)

In [ ]:
LogisticRegression(C=0.0001, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=50000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [ ]:
preds = m2.predict(valid_ngram_doc_matrix)
(preds.T==valid_labels).mean()


**Log-count ratio** r

In [ ]:
train_ngram_doc_matrix.sign()

In [ ]:
valid_ngram_doc_matrix.sign()

In [ ]:
positive = y.c2i['positive']
negative = y.c2i['negative']
positive, negative

In [ ]:
k=260374

In [ ]:
pos = (y.items == positive)[:k]
neg = (y.items == negative)[:k]


In [ ]:
xx = x

In [ ]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [ ]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [ ]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [ ]:
r = np.log(pr1/pr0)
r

In [ ]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())
b

In [ ]:
np.exp(r)


In [ ]:
xx = x[:k]

In [ ]:
len(r)

In [ ]:
xx.shape

In [0]:
x_nb = xx @ r + b
#m = LogisticRegression(dual=True, C=0.1)
#m.fit(x_nb, y.items);



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: ignored

In [0]:
val_x_nb = val_x @ r + b
preds = m.predict(val_x_nb)
(preds.T==valid_labels).mean()

NotFittedError: ignored


References

    Baselines and Bigrams: Simple, Good Sentiment and Topic Classification. Sida Wang and Christopher D. Manning pdf

